In [196]:
import numpy as np
import hashlib
import random
import string
import hmac
import pandas as pd
from tqdm.notebook import tqdm
import os
import re

import config
import get_game_data

tqdm.pandas()

import warnings
warnings.filterwarnings("ignore")

### Load data

In [197]:
multipliers = get_game_data.get_all_multipliers(config.first_game,config.current_game)
multipliers_chronological_order = multipliers[::-1]
multiplier_df = pd.DataFrame({'game_no': range(1,len(multipliers_chronological_order)+1), 'multiplier': multipliers_chronological_order})

multipler_dataset_name = "raw_multipler_" + ".csv"
raw_multipler_data_path = os.path.join("data",multipler_dataset_name)
multiplier_df.to_csv(raw_multipler_data_path,index=False)

### Train Test Split
Note that each row is a game played.

In [213]:
raw_multiplier_df = pd.read_csv(raw_multipler_data_path)
multipler_df = raw_multiplier_df.copy()

In [214]:
#train test split
train_test_split_index = round(len(multiplier_df.index) * config.train_pct)

testing_df = multiplier_df.loc[train_test_split_index:].copy()
training_df = multiplier_df.loc[:train_test_split_index-1].copy()

In [215]:
#create results folder
experiment_results_directory = "loss_sequence_playing"
results_path = os.path.join("data",experiment_results_directory)
try:    
    os.mkdir(results_path)
except FileExistsError:
    pass

### Multiple loss play
After multiple red games, a green game must occur.

In [216]:
#experiment parameters
loss_level = 2
multiplier_checkout_level = 2
bet_size = 5
death_stop = 2
num_loss = 4 #number of losses in a row
loss_counter = 0

In [217]:
#True if lost game, False if won game
training_df["loss_flag"] = training_df["multiplier"] < loss_level

In [218]:
#create game windows
training_windows = training_df

,game_no,multiplier,loss_flag
0,1,2.19,False
1,2,2.80,False
2,3,6.55,False
3,4,1.10,True
4,5,1.06,True
...,...,...,...
1538097,1538098,1.10,True
1538098,1538099,3.72,False
1538099,1538100,17.31,False
1538100,1538101,1.38,True


In [220]:
num_games = training_df.shape[0]//num_loss

In [222]:
[training_df.loc[game_idx*num_loss:(game_idx+1)*num_loss] for game_idx in range(num_games)]

In [203]:
#True if lost game, False if won game
training_df["loss_flag"] = training_df["multiplier"] < loss_level

for num_loss in number_of_losses_in_a_row:
    #find game sequences with the appropriate number of losses in a row
    training_df["cumsum"] = (~training_df["loss_flag"]).cumsum()

    #find where cumsum doesn't change for num_games lost in a row
    cumsum_value_counts = training_df["cumsum"].value_counts()
    appropriate_cumsum = cumsum_value_counts[cumsum_value_counts >= num_loss]
    game_sequences = training_df[training_df["cumsum"].isin(appropriate_cumsum.index)].query("loss_flag == True")


    break

In [211]:
appropriate_cumsum

516735    21
455783    21
719132    21
579838    19
394652    19
          ..
272412     2
31347      2
204139     2
225544     2
635726     2
Name: cumsum, Length: 384309, dtype: int64

In [193]:
#find game sequences with the appropriate number of losses in a row
training_df["cumsum"] = (~training_df["loss_flag"]).cumsum()
#find where cumsum doesn't change for num_games lost in a row
cumsum_value_counts = training_df["cumsum"].value_counts()
appropriate_cumsum = cumsum_value_counts[cumsum_value_counts > number_of_losses_in_a_row]
game_sequences = training_df[training_df["cumsum"].isin(appropriate_cumsum.index)].query("loss_flag == True")

#extract game after loss sequence
game_after_loss_sequence = game_sequences.groupby(["cumsum"])["game_no"].max().reset_index()
game_after_loss_sequence["sequential_game_no"] = game_after_loss_sequence["game_no"] + 1
game_after_loss_sequence = training_df[training_df["game_no"].isin(game_after_loss_sequence["sequential_game_no"])]

# #append loss sequences to game after loss sequence 
# #note games after loss sequence are games to bet on
game_sequences = game_sequences.append(game_after_loss_sequence, ignore_index=True).sort_values(by="cumsum")

In [195]:
game_after_loss_sequence["loss_flag"].unique()

array([False])

In [167]:
#game played stats
num_of_games_played = game_after_loss_sequence["game_no"].nunique()
num_of_games_won = game_after_loss_sequence[game_after_loss_sequence["multiplier"] > multiplier_checkout_level]["game_no"].nunique()
pct_games_won = num_of_games_won/num_of_games_played*100
print(pct_games_won)


99.51772535516155


In [168]:
#play game after loss sequence
#place a bet at the specific checkout level and bet amount
def force_game_play(loss_flag, multiplier_checkout_level, bet_size):
    if not(loss_flag):
        #game is above 2
        game_result = multiplier_checkout_level * bet_size
    else:
        #if game is a loss (below 2)
        game_result = 0

    return game_result

In [169]:
#calculate game result
game_after_loss_sequence["game_result"] = game_after_loss_sequence.progress_apply(lambda x : force_game_play(x["loss_flag"],multiplier_checkout_level, bet_size), axis=1)

#P&L - positive is good, negative is bad
capital_injected = game_after_loss_sequence["game_no"].nunique() * bet_size
games_result = game_after_loss_sequence["game_result"].sum()
loss_gain = games_result - capital_injected

  0%|          | 0/198642 [00:00<?, ?it/s]

In [170]:
loss_gain

993210